In [65]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sys
import os
import torch
import warnings
import yaml
from easydict import EasyDict as edict
from pprint import pprint

%load_ext autoreload
%autoreload 2
%matplotlib inline

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (16, 3)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [169]:
from flask.scaffold import F
import yfinance as yf
from pandas_datareader import data as pdr

yf.pdr_override()

from datetime import datetime as dt

import yaml


class StockAnalyer:
    def __init__(self, cfg_name="cfg-quant", cfg_dir="./configs"):
        with open(f"{cfg_dir}/{cfg_name}.yaml") as file:
            cfg = yaml.load(file, Loader=yaml.FullLoader)
            self.cfg = edict(cfg)
        self.__load__()

        companies = []
        stats = []
        for i, (company, ticker) in enumerate(self.data_dic.items()):
            if i >= 1:
                break
            print(f">>>>>>>>>>>>>> {company} <<<<<<<<<<<<<<<")
            stat = self.analyze_stat(ticker)
            stats.append(stat)
            companies.append(company)

        print(stats)
        print(companies)
        df = pd.DataFrame(stats).T
        df.columns = companies
        display(df)

        stats_explanation = self.cfg.stats_explanation

    def __load__(self):
        tickers = self.cfg.tickers

        data_dic = {}
        for key, ticker in tickers.items():
            data_dic[key] = yf.Ticker(ticker)

        self.data_dic = data_dic

    def analyze_stat(self, ticker):
        stat = {}
        info = ticker.info
        bal = ticker.balance_sheet
        inc = ticker.income_stmt
        fin = ticker.financials
        cash = ticker.cashflow

        # info
        beta = info["beta"]
        market_cap = info["marketCap"]

        # financials
        ebit = fin.loc["EBIT"][0]
        ebitda = fin.loc["EBITDA"][0]

        # balance
        total_debt = bal.loc["Total Debt"][0]
        total_asset = bal.loc["Total Assets"][0]
        equity = total_asset - total_debt
        # income
        net_income = inc.loc["Net Income"][0]

        # cash
        total_cash = cash.loc["Free Cash Flow"][0]

        # * Statistics
        # basic
        PER = market_cap / net_income
        stat["PER"] = PER
        print(f"PER: {PER:.2f}")

        # good company
        net_operating_profit_after_tax = inc.loc["Net Income"][0]
        net_tangible_assets = bal.loc["Net Tangible Assets"][0]
        non_current_asset = bal.loc["Total Non Current Assets"][0]
        working_capital = bal.loc["Working Capital"][0]
        modified_ROIC = net_operating_profit_after_tax / (
            non_current_asset + working_capital
        )
        print(f"modified_ROIC: {modified_ROIC:.2f}")
        stat["modified_ROIC"] = modified_ROIC

        nopat4 = inc.loc["Net Income"].sum()
        non_current_asset4 = bal.loc["Total Non Current Assets"].sum()
        working_capital4 = bal.loc["Working Capital"].sum()

        modified_ROIC4 = nopat4 / (non_current_asset4 + working_capital4)
        print(f"modified_ROIC4: {modified_ROIC4:.2f}")
        stat["modified_ROIC4"] = modified_ROIC4

        # good earning
        ev = market_cap - total_debt + total_cash
        earning_rate = ebit / ev
        print(f"earning_rate: {earning_rate:.2f}")
        stat["EBIT/EV"] = earning_rate
        stat["EBITDA/EV"] = ebitda / ev

        # good quality(gross profitability)
        total_revenue = inc.loc["Total Revenue"][0]
        gross_profit = inc.loc["Gross Profit"][0]

        GPA = gross_profit / equity
        print(f"GPA: {GPA:.2f}")
        stat["GPA"] = GPA

        ROA = net_income / total_asset
        print(f"ROA: {ROA:.2f}")
        stat["ROA"] = ROA

        total_asset4 = bal.loc["Total Assets"].sum()
        net_income4 = inc.loc["Net Income"].sum()
        ROA4 = net_income4 / total_asset4
        print(f"ROA4: {ROA4:.2f}")
        stat["ROA4"] = ROA4

        MG = inc.loc["Gross Profit"][0] / inc.loc["Total Revenue"][0]
        print(f"MG: {MG:.2f}")
        stat["MG"] = MG

        MS = inc.loc["Gross Profit"].mean() / inc.loc["Gross Profit"].std()
        print(f"MS: {MS:.2f}")
        stat["MS"] = MS

        operating_cash_flow = cash.loc["Operating Cash Flow"][0]
        # capex = cash.loc["Capital Expenditure"][0]
        PCR = market_cap / operating_cash_flow
        print(f"PCR: {PCR:.2f}")
        stat["PCR"] = PCR

        ROE = net_income / equity
        stat["ROE"] = ROE

        PBR = PER * ROE
        print(f"PBR: {PBR:.2f}")
        stat["PBR"] = PBR

        PSR = market_cap / total_revenue
        print(f"PSR: {PSR:.2f}")
        stat["PSR"] = PSR

        return stat

    def get_data_dic(self):
        return self.data_dic

    def forward(self):
        pass


stock_analyer = StockAnalyer()

>>>>>>>>>>>>>> asml <<<<<<<<<<<<<<<
PER: 48.34
modified_ROIC: 0.33
modified_ROIC4: 0.28
earning_rate: 0.02
GPA: 0.40
ROA: 0.20
ROA4: 0.17
MG: 0.51
MS: 3.43
PCR: 69.62
PBR: 10.73
PSR: 13.75
[{'PER': 48.340926163031, 'modified_ROIC': 0.3309997128717888, 'modified_ROIC4': 0.28413268566252875, 'EBIT/EV': 0.02446285904163589, 'EBITDA/EV': 0.02642228208563691, 'GPA': 0.40016248701377743, 'ROA': 0.1961834449102171, 'ROA4': 0.17120763657487534, 'MG': 0.5129488179690477, 'MS': 3.4300015035930556, 'PCR': 69.61540952198993, 'ROE': 0.2219051743904614, 'PBR': 10.727101650403812, 'PSR': 13.750549565179528}]
['asml']


,asml
PER,48.340926
modified_ROIC,0.331000
modified_ROIC4,0.284133
EBIT/EV,0.024463
EBITDA/EV,0.026422
GPA,0.400162
ROA,0.196183
ROA4,0.171208
MG,0.512949
MS,3.430002
